In [4]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchvision.datasets import CocoDetection
from tqdm import tqdm

from architectures.basic import BoundingBoxModel as BasicBoundingBoxModel
from architectures.bigger_basic import BoundingBoxModel as BiggerBasicBoundingBoxModel
from functions.loop_management import EarlyStopping
from functions.losses import CIoULoss
from functions.mapping import extract_bboxes
from functions.plotting import plot_losses

print(torch.__version__)
print(torch.cuda.is_available())

2.3.1
True


In [5]:
root = os.path.join("data", "test_images")
ann_file = os.path.join("data", "test_coco_annotations.json")
transform = transforms.Compose([
    transforms.ToTensor()
])
dataset = CocoDetection(
    root=root,
    annFile=ann_file,
    transform=transform
)
test_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
    prefetch_factor=16
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
cuda


In [11]:
basic_model = BasicBoundingBoxModel().to(device)
bigger_basic_model = BiggerBasicBoundingBoxModel().to(device)
bigger_basic_ciou_model = BiggerBasicBoundingBoxModel().to(device)

In [12]:
basic_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_1_basic.pt"), map_location=device)["model_state_dict"])
basic_model.eval()
bigger_basic_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_2_bigger_basic.pt"), map_location=device)["model_state_dict"])
bigger_basic_model.eval()
bigger_basic_ciou_model.load_state_dict(torch.load(os.path.join("checkpoints", "checkpoint_3_bigger_basic_ciou.pt"), map_location=device)["model_state_dict"])
bigger_basic_ciou_model.eval()

BoundingBoxModel(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=57600, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)

In [13]:
def test_model(model: nn.Module) -> list:
    results = []

    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            outputs = model(images)
            batch_results = extract_bboxes(outputs)
            results.extend(batch_results)
            
    return results

In [14]:
basic_model_results = test_model(basic_model)
bigger_basic_model_results = test_model(bigger_basic_model)
bigger_basic_ciou_model_results = test_model(bigger_basic_ciou_model)

IndexError: too many indices for tensor of dimension 1